### Load Libraries

In [74]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os

### Load Raw Training Data

In [2]:
inpData = []
labels = []
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
        

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
index = 0
while index + 1 < len(inpData):
    if inpData[index + 1] > maxThreshold:
        target.append(1)
    elif inpData[index + 1] < minThreshold:
        target.append(-1)
    else:
        target.append(0)
    index += 1

In [ ]:
print(target)

### Preprocessing Input Data

In [4]:
cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1
    

In [5]:
cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

In [6]:
#len(inpData)
X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.25, random_state=42)

In [17]:
cpuUsageData.shape
#np.shape(cpuUsage)

(5035, 50, 1)

In [18]:
cpuScalingCategorical.shape

(5035, 3)

In [69]:
model = Sequential()
#model.add(SpatialDropout1D(0.3))
model.add(Flatten(batch_input_shape = (None,50,1)))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(25, activation = 'relu'))
model.add(Dropout(0.3, noise_shape=None, seed=None))
#model.add(SpatialDropout1D(0.3))
model.add(Dense(12,activation = 'relu'))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(6,activation = 'relu'))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
#model.summary()

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_53 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 25)                1275      
_________________________________________________________________
dropout_54 (Dropout)         (None, 25)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 12)                312       
_________________________________________________________________
dropout_55 (Dropout)         (None, 12)                0         
__________

In [75]:
filepath="3OutputDNNBestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
if os.path.isfile("3OutputDNNBestWeights.hdf5"):
    #print("Hey")
    model.load_weights("3OutputDNNBestWeights.hdf5")

In [76]:
history = model.fit(X_train, y_train, epochs = 50,validation_data = [X_test, y_test],verbose = 0, callbacks = callbacks_list)


Epoch 00001: val_acc improved from -inf to 0.93328, saving model to 3OutputDNNBestWeights.hdf5

Epoch 00002: val_acc did not improve from 0.93328

Epoch 00003: val_acc did not improve from 0.93328

Epoch 00004: val_acc did not improve from 0.93328

Epoch 00005: val_acc did not improve from 0.93328

Epoch 00006: val_acc did not improve from 0.93328

Epoch 00007: val_acc did not improve from 0.93328

Epoch 00008: val_acc did not improve from 0.93328

Epoch 00009: val_acc did not improve from 0.93328

Epoch 00010: val_acc did not improve from 0.93328

Epoch 00011: val_acc did not improve from 0.93328

Epoch 00012: val_acc did not improve from 0.93328

Epoch 00013: val_acc did not improve from 0.93328

Epoch 00014: val_acc did not improve from 0.93328

Epoch 00015: val_acc did not improve from 0.93328

Epoch 00016: val_acc did not improve from 0.93328

Epoch 00017: val_acc did not improve from 0.93328

Epoch 00018: val_acc did not improve from 0.93328

Epoch 00019: val_acc did not improve

In [ ]:
res = model.predict(X_test,batch_size = 1)

In [ ]:
print(res)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])